In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#Required Imports
from google.cloud import bigquery
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Setup your client project id
PROJECT_ID = 'aqueous-nebula-343009'
client = bigquery.Client(project = PROJECT_ID)

In [5]:
def load_data_from_ref_object(client, dataset_name, table_name):
    dataset_ref = client.dataset(dataset_name)
    dataset = client.get_dataset(dataset_ref)
    print("List of all Tables in Dataset --")
    tables = list(client.list_tables(dataset))
    for t in tables:
        print(t.table_id)
        
    table_ref =  dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    
    print("Schema of the Table --")
    print(table.schema)
    data = client.list_rows(table).to_dataframe()
    return data

In [6]:
#Give your Dataset and Table Name
data = load_data_from_ref_object(client, "Churn", "Churn_Modelling")
data.head()

In [7]:
def load_dataset_from_query(client, query_):
    job = client.query(query_)
    return job.to_dataframe()

In [8]:
#project_id.datasetname.table_name
data = load_dataset_from_query(client, "SELECT * FROM aqueous-nebula-343009.Churn.Churn_Modelling")
data.head()

**Comparing Exited and Non-Exited**

In [10]:
data = load_dataset_from_query(client, "SELECT Exited, COUNT(Exited) AS COUNT FROM aqueous-nebula-343009.Churn.Churn_Modelling GROUP BY Exited")
data.head()

In [13]:
plt.figure(figsize = (8,6))
sns.barplot(data['Exited'], data['COUNT'])

Comparison of Estimated Salary between Exited and Non-Exited

In [15]:
data = load_dataset_from_query(client, "SELECT Exited, AVG(EstimatedSalary) AS Salary FROM aqueous-nebula-343009.Churn.Churn_Modelling GROUP BY Exited")
data.head()

In [16]:
plt.figure(figsize = (8,6))
sns.barplot(data['Exited'], data['Salary'])

Comparison Between Exited and Country


In [22]:
data = load_dataset_from_query(client, "SELECT Geography, Exited, COUNT(Exited) AS Count FROM aqueous-nebula-343009.Churn.Churn_Modelling GROUP BY Geography, Exited ")
data.head()

Exited, Gender, and Geography

Filter by Exiting

In [27]:
data = load_dataset_from_query(client, "SELECT Geography, Gender, COUNT(Exited) AS Count FROM aqueous-nebula-343009.Churn.Churn_Modelling GROUP BY Geography, Gender, Exited HAVING Exited = 1 ")
data.head()

In [29]:
plt.figure(figsize = (8,6))
sns.barplot(x = "Geography", y = "Count", hue = "Gender", data = data)

Filter by Male

In [30]:
data = load_dataset_from_query(client, "SELECT Geography, Gender, COUNT(Exited) AS Count FROM aqueous-nebula-343009.Churn.Churn_Modelling GROUP BY Geography, Gender, Exited HAVING Exited = 0 ")
data.head()

In [31]:
plt.figure(figsize = (8,6))
sns.barplot(x = "Geography", y = "Count", hue = "Gender", data = data)